In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat May 23 16:54:55 2020

@author: Nick
"""
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import re 
from functions import *

intent, unique_intent, sentences = load_dataset("data3.csv")

print(sentences[:5])
nltk.download("stopwords")
nltk.download("punkt")

for i in range(len(sentences)):
  sentences[i] = re.sub(r'^https?:\/\/.*[\r\n]*', '', str(sentences[i]), flags=re.MULTILINE)

cleaned_words = cleaning(sentences)   
stemmer = LancasterStemmer()


word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

encoded_doc = encoding_doc(word_tokenizer, cleaned_words)
padded_doc = padding_doc(encoded_doc, max_length)
padded_doc[:5]
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
output_tokenizer.word_index
encoded_output = encoding_doc(output_tokenizer, intent)
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

encoded_output.shape

output_one_hot = one_hot(encoded_output)





 
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred

def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.1)
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))


model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs=42 , batch_size = 15, validation_data = (val_X, val_Y), callbacks = [checkpoint])



model = load_model("model.h5")















                                            Sentence Intent
0  Hi Criag,\r\n\r\nThank you for reaching out.  ...   pass
1                     Thanks Craig\r\nWe will pass f   pass
2  HI Craig \xe2\x80\x93 this one is a pass for m...   pass
3  Hi Craig,\r\n\r\nIt\xe2\x80\x99s nice to meet ...   pass
4  Appreciate heads up on it, I will need to pass...   pass
["Hi Criag,\\r\\n\\r\\nThank you for reaching out.  However, this opportunity is completely\\r\\noutside our fund's focus on consumer brands, thus we will pass.\\r\\n\\r\\nBest of luck with the closing!\\r\\n\\r\\nSarah\\r\\n\\xe1\\x90\\xa7\\r\\n\\r\\n", 'Thanks Craig\\r\\nWe will pass f', 'HI Craig \\xe2\\x80\\x93 this one is a pass for me, outside my knowledge base', 'Hi Craig,\\r\\n\\r\\nIt\\xe2\\x80\\x99s nice to meet you.  Unfortunately, we\\xe2\\x80\\x99ll pass on this.  I hope you stay\\r\\nsafe and healthy.\\r\\n\\r\\nBest regards,\\r\\nJohn\\r\\n\\r\\nO', 'Appreciate heads up on it, I will need to pass on this one.  Thx\\r\

In [ ]:
def get_final(pred, classes):
  predictions = pred[0]
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
  return classes[0]


In [ ]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred

In [ ]:
text = """Hey Mate, \r\n\r\n\r\nI\xe2\x80\x99m really interested Craig, let\xe2\x80\x99s set up a time to meet up and discu"""
pred = predictions(text)
get_final_output(pred, unique_intent)
get_final(pred, unique_intent)

NameError: ignored

In [ ]:
##Creating a csv file with the interested/pass values for every email 
import csv 

f  = open("ID_cmckeon@cbmpartners.csv")
csv_f  = csv.reader(f)
classifications = {}


def add_all(a):
  string = ''
  for i in range(len(a)):
    string = string+a[i]
  return string



for row in csv_f:
  emailnum = row[-1]
  print("Email-num:")
  print(emailnum)
  print("Dingo")
  msg = row[1]
  pre  = predictions(msg)
  classifications[emailnum] = get_final(pre, unique_intent)


filename = "CLASSIFICATIONS_CMCKEON@CBMPARTNERS.csv"
with open(filename,'w') as document:
  document.write("Email, Classification\n")
  for key in classifications.keys():
    document.write("%s,%s\n"%(key,classifications[key]))











Email-num:
jibreel@petramdata.com
Dingo
Things are exceedingly busy for me and my team right now.  I\xe2\x80\x99d suggest you\r\nmay want to talk to my business partner Dhani Jones (cc\xe2\x80\x99d here).\r\n\r\n\r\n\r\nJibreel\r\n\r\n\r\n\r\n*From:* Craig mckeon 
['things', 'are', 'exceedingly', 'busy', 'for', 'me', 'and', 'my', 'team', 'right', 'now', 'i', 'xe2', 'x80', 'x99d', 'suggest', 'you', 'r', 'nmay', 'want', 'to', 'talk', 'to', 'my', 'business', 'partner', 'dhani', 'jones', 'cc', 'xe2', 'x80', 'x99d', 'here', 'r', 'n', 'r', 'n', 'r', 'n', 'r', 'njibreel', 'r', 'n', 'r', 'n', 'r', 'n', 'r', 'n', 'from', 'craig', 'mckeon']
